In [1]:
!git clone https://github.com/aayugay99/efficient_transformers_on_transactional_data 

Cloning into 'efficient_transformers_on_transactional_data'...
remote: Enumerating objects: 171, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 171 (delta 10), reused 13 (delta 5), pack-reused 135
Receiving objects: 100% (171/171), 72.95 KiB | 9.12 MiB/s, done.
Resolving deltas: 100% (91/91), done.


In [2]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 11.2 MB/s eta 0:00:00


In [3]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB 24.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.5 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=357f97791f7ec1d90f81b63f26e051664faf6ea5cc9ea0170ad6f00c75285c3f
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [4]:
%cd /content/efficient_transformers_on_transactional_data

/content/efficient_transformers_on_transactional_data


In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import KBinsDiscretizer

import torch
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

from dataset import TransactionDataset, transaction_collate_fn

from torchmetrics.classification import MulticlassF1Score, Accuracy

from sklearn.model_selection import train_test_split

import time

import wandb

In [6]:
!sh /content/efficient_transformers_on_transactional_data/data_download

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  239M  100  239M    0     0  14.4M      0  0:00:16  0:00:16 --:--:-- 16.3M
Archive:  age-prediction-nti-sbebank-2019.zip
  inflating: data/sberbank/test.csv  
  inflating: data/sberbank/small_group_description.csv  
  inflating: data/sberbank/train_target.csv  
  inflating: data/sberbank/transactions_train.csv  
  inflating: data/sberbank/transactions_test.csv  
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 8979k  100 8979k    0     0  3488k      0  0:00:02  0:00:02 --:--:-- 3487k


In [7]:
df = pd.read_csv('data/rosbank/train.csv')
df['TRDATETIME'] = pd.to_datetime(df['TRDATETIME'], format='%d%b%y:%H:%M:%S')
df = df.rename(columns={'cl_id':'client_id', 'MCC':'small_group', 'amount':'amount_rur'})
df.head()

,PERIOD,client_id,small_group,channel_type,currency,TRDATETIME,amount_rur,trx_category,target_flag,target_sum
0,01/10/2017,0,5200,NaN,810,2017-10-21 00:00:00,5023.0,POS,0,0.0
1,01/10/2017,0,6011,NaN,810,2017-10-12 12:24:07,20000.0,DEPOSIT,0,0.0
2,01/12/2017,0,5921,NaN,810,2017-12-05 00:00:00,767.0,POS,0,0.0
3,01/10/2017,0,5411,NaN,810,2017-10-21 00:00:00,2031.0,POS,0,0.0
4,01/10/2017,0,6012,NaN,810,2017-10-24 13:14:24,36562.0,C2C_OUT,0,0.0


In [8]:
mcc_to_id = {mcc: i+1 for i, mcc in enumerate(df['small_group'].unique())}

df['amount_rur_bin'] = 1 + KBinsDiscretizer(10, encode='ordinal', subsample=None).fit_transform(df[['amount_rur']]).astype('int')
df['small_group'] = df['small_group'].map(mcc_to_id)

In [9]:
df

,PERIOD,client_id,small_group,channel_type,currency,TRDATETIME,amount_rur,trx_category,target_flag,target_sum,amount_rur_bin
0,01/10/2017,0,1,NaN,810,2017-10-21 00:00:00,5023.00,POS,0,0.0,9
1,01/10/2017,0,2,NaN,810,2017-10-12 12:24:07,20000.00,DEPOSIT,0,0.0,10
2,01/12/2017,0,3,NaN,810,2017-12-05 00:00:00,767.00,POS,0,0.0,6
3,01/10/2017,0,4,NaN,810,2017-10-21 00:00:00,2031.00,POS,0,0.0,8
4,01/10/2017,0,5,NaN,810,2017-10-24 13:14:24,36562.00,C2C_OUT,0,0.0,10
...,...,...,...,...,...,...,...,...,...,...,...
490508,01/04/2017,10176,2,type1,810,2017-04-24 14:05:26,600.00,WD_ATM_ROS,1,405.0,5
490509,01/06/2017,10171,4,type1,810,2017-06-06 00:00:00,132.00,POS,0,0.0,2
490510,01/02/2017,10167,51,type1,810,2017-02-03 00:00:00,1000.00,POS,1,280428.2,7
490511,01/06/2017,10163,39,type1,810,2017-06-08 00:00:00,100.00,POS,0,0.0,2


In [10]:
clients_train, clients_val_test = train_test_split(df["client_id"].unique(), test_size=0.2, random_state=42)
clients_val, clients_test = train_test_split(clients_val_test, test_size=0.5, random_state=42)

train_ds = TransactionDataset(
    df[lambda x: x["client_id"].isin(clients_train)], 
    id_col="client_id", 
    dt_col="TRDATETIME", 
    cat_cols=["small_group", "amount_rur_bin"],
    min_length=20,
    max_length=100
)

val_ds = TransactionDataset(
    df[lambda x: x["client_id"].isin(clients_val)], 
    id_col="client_id", 
    dt_col="TRDATETIME", 
    cat_cols=["small_group", "amount_rur_bin"],
    min_length=20,
    max_length=100
)

test_ds = TransactionDataset(
    df[lambda x: x["client_id"].isin(clients_test)], 
    id_col="client_id", 
    dt_col="TRDATETIME", 
    cat_cols=["small_group", "amount_rur_bin"],
    min_length=20,
    max_length=100
)

628 sequences were filtered
77 sequences were filtered
70 sequences were filtered


In [11]:
!pip install linear-attention-transformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for product-key-memory: filename=product_key_memory-0.1.10-py3-none-any.whl size=3053 sha256=9884e8f78f106cfc32dbbce73b72380e50d37913243570aac41194f26aeea70e
  Stored in directory: /root/.cache/pip/wheels/2c/97/82/f94ef75772952e1eaec19864fc840620ec94d9ac7c9c0b6823
  Created wheel for axial-positional-embedding: filename=axial_positional_embedding-0.2.1-py3-none-any.whl size=2882 sha256=af0636d89117e791387ea77a49cf11204f0d09eb5c8751823daa6095103f6e86
  Stored in directory: /root/.cache/pip/wheels/b1/cb/39/7ce7ff2d2fd37cfe1fe7b3a3c43cf410632b2ad3b3f3986d73
Successfully built product-key-memory axial-positional-embedding


In [12]:
from linear_attention_transformer import LinearAttentionTransformer, LinearAttentionTransformerLM

In [13]:
model = LinearAttentionTransformer(
    dim = 512,
    depth = 1,
    heads = 8,
    causal = True,
    max_seq_len = 100,
    dim_head = 64
)

x = torch.randn(1, 2048, 512)
model(x).shape

torch.Size([1, 2048, 512])

In [14]:
import math
class TransactionEncoder(nn.Module):
    def __init__(self, feature_embeddings: dict[str, tuple[int, int]], linear_proj: int=None):
        super().__init__()
        
        self.feature_embeddings = feature_embeddings
        self.embeddings = nn.ModuleDict({key: nn.Embedding(vocab, dim) for key, (vocab, dim) in feature_embeddings.items()})
        
        if linear_proj is not None:
            self.embedding_dim = linear_proj
            self.linear_proj = nn.Linear(sum([dim for key, (vocab, dim) in feature_embeddings.items()]), linear_proj)
        else:
            self.embedding_dim = sum([dim for key, (vocab, dim) in feature_embeddings.items()])
            self.linear_proj = nn.Identity()

    def forward(self, x: torch.Tensor, device: str="cpu") -> torch.Tensor:
        embeddings = [self.embeddings[key](x[key].to(device)) for key in self.feature_embeddings]
        proj = self.linear_proj(torch.cat(embeddings, dim=2))
        return proj


class Head(nn.Module):
    def __init__(self, embedding_dim: int, hidden_dim: int, vocab_size: int):
        super().__init__()

        self.head = nn.Sequential(
            nn.Linear(embedding_dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, vocab_size),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.head(x)


class PositionalEncoding(nn.Module):
    def __init__(self, embedding_dim: int, dropout: float=0.1, max_len: int=1000):
        super().__init__()
        self.dropout = nn.Dropout(dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embedding_dim, 2) * (-math.log(10000.0) / embedding_dim))
        pe = torch.zeros(1, max_len, embedding_dim)
        pe[0, :, 0::2] = torch.sin(position * div_term)
        pe[0, :, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Arguments:
            x: Tensor, shape ``[batch_size, seq_len, embedding_dim]``
        """
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)


class LinearTransformerModel(nn.Module):
    def __init__(
            self, 
            feature_embeddings: dict[str, tuple[int, int]], 
            linear_proj: int=None,
            n_head: int=8, 
            dim_feedforward: int=128, 
            dropout: float=0.1, 
            num_layers: int=6, 
            head_hidden: int=128,
            max_len: int=1000,
        ):
        super().__init__()

        self.transaction_encoder = TransactionEncoder(feature_embeddings, linear_proj=linear_proj)
        self.embedding_dim = self.transaction_encoder.embedding_dim
        self.cat_cols = list(feature_embeddings.keys())
        self.num_classes_dict = {key: num_classes for key, (num_classes, _) in feature_embeddings.items()}
        
        self.pos_emb = PositionalEncoding(self.embedding_dim, dropout, max_len)

        self.transformer_encoder = LinearAttentionTransformer(
            dim = self.embedding_dim, 
            depth = num_layers,
            heads = n_head, 
            ff_dropout = dropout,
            causal = True,
            dim_head = 16,
            max_seq_len = 100
        )
        
        self.heads = nn.ModuleDict({
            key: Head(
                self.embedding_dim, 
                head_hidden, 
                num_classes
            ) for key, num_classes in self.num_classes_dict.items()
        })

    def forward(self, x: torch.Tensor, device: str="cpu") -> torch.Tensor:
        N, S = x[self.cat_cols[0]].shape
        embeddings = self.transaction_encoder(x, device=device)
        embeddings = self.pos_emb(embeddings)
        
        attn_mask = self.generate_square_subsequent_mask(S).to(device)
        padding_mask = self.generate_padding_mask(x[self.cat_cols[0]]).to(device)
        embeddings = self.transformer_encoder(embeddings, input_mask=padding_mask)

        logits = {key: self.heads[key](embeddings) for key in self.cat_cols}
        return logits

    @staticmethod
    def generate_square_subsequent_mask(sz: int) -> torch.Tensor:
        return torch.triu(torch.full((sz, sz), True), diagonal=1).bool()
    
    @staticmethod
    def generate_padding_mask(x: torch.Tensor) -> torch.Tensor:
        return torch.where(x == 0, True, False).bool()

In [15]:
from tqdm.notebook import tqdm

In [16]:
import os

def train_epoch(model, optimizer, dataloader, warmup=10, device="cuda"):
    model.train()
    model.to(device)

    metrics = {
        key: {
            "f1_score": MulticlassF1Score(
                num_classes=num_classes, 
                average="weighted", 
                ignore_index=0
            ), 
            "accuracy": Accuracy(
                task="multiclass", 
                num_classes=num_classes, 
                ignore_index=0
            )
        } for key, num_classes in model.num_classes_dict.items()
    }

    loss_epoch = 0
    count = 0 
    for batch_dict in tqdm(dataloader):
        logits_dict = model(batch_dict, device=device)

        loss = 0
        for key, logits in logits_dict.items():
            y = batch_dict[key][:, warmup + 1:].to(device)
            logits_pred = logits[:, warmup: -1].permute(0, 2, 1)    # B x C x T

            loss += nn.functional.cross_entropy(logits_pred, y, ignore_index=0)
            
            y_pred = logits_pred.argmax(dim=1).to("cpu")
            metrics[key]["f1_score"].update(y_pred, y.to("cpu"))
            metrics[key]["accuracy"].update(y_pred, y.to("cpu"))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cur_count =  torch.sum((y != 0).float()).item()
        loss_epoch += loss.item() * cur_count
        count += cur_count

    return loss_epoch / count, {feature: {m: v.compute().item() for m, v in results.items()} for feature, results in metrics.items()}


def eval_epoch(model, dataloader, warmup=10, device="cuda"):
    model.eval()
    model.to(device)

    metrics = {
        key: {
            "f1_score": MulticlassF1Score(
                num_classes=num_classes, 
                average="weighted", 
                ignore_index=0
            ), 
            "accuracy": Accuracy(
                task="multiclass", 
                num_classes=num_classes, 
                ignore_index=0
            )
        } for key, num_classes in model.num_classes_dict.items()
    }

    with torch.no_grad():
        loss_epoch = 0
        count = 0 
        for batch_dict in dataloader:
            logits_dict = model(batch_dict, device=device)

            loss = 0
            for key, logits in logits_dict.items():
                y = batch_dict[key][:, warmup + 1:].to(device)
                logits_pred = logits[:, warmup: -1].permute(0, 2, 1)
        
                loss += nn.functional.cross_entropy(logits_pred, y, ignore_index=0)

                y_pred = logits_pred.argmax(dim=1).to("cpu")
                metrics[key]["f1_score"].update(y_pred, y.to("cpu"))
                metrics[key]["accuracy"].update(y_pred, y.to("cpu"))

            cur_count = torch.sum((y != 0).float()).item()
            loss_epoch += loss.item() * cur_count
            count += cur_count    

    return loss_epoch / count, {feature: {m: v.compute().item() for m, v in results.items()} for feature, results in metrics.items()}


def train_model(model, optimizer, dataloaders, n_epochs, warmup=10, device="cuda", save_path="./"):
    if not os.path.exists(save_path):
        os.mkdir(save_path)

    best_loss = float("inf")
    for epoch in tqdm(range(n_epochs)):
        train_start = time.perf_counter()
        train_loss, train_metrics = train_epoch(model, optimizer, dataloaders["train"], warmup, device)
        train_end = time.perf_counter()
        val_loss, val_metrics = eval_epoch(model, dataloaders["val"], warmup, device)
        val_end = time.perf_counter()

        wandb.log({
            "Epoch": epoch+1,
            "Train time": train_end - train_start,
            "Train loss": train_loss,
            "Train metrics": train_metrics,
            "Val time": val_end - train_end,
            "Val metrics": val_metrics,
            "Val loss": val_loss
        })

        if val_loss < best_loss:
             best_loss = val_loss
             torch.save(model, os.path.join(save_path, "best_model.pt"))

    model = torch.load(os.path.join(save_path, "best_model.pt"))

    test_start = time.perf_counter()
    test_loss, test_metrics = eval_epoch(model, dataloaders["test"], warmup, device)
    test_end = time.perf_counter()

In [17]:
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, collate_fn=transaction_collate_fn)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False, collate_fn=transaction_collate_fn)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False, collate_fn=transaction_collate_fn)

In [18]:
transformer = LinearTransformerModel(
    feature_embeddings={"small_group": (345, 64), "amount_rur_bin": (11, 64)}, 
    linear_proj=64,
    n_head=8, 
    dim_feedforward=128, 
    dropout=0.1, 
    num_layers=6, 
    head_hidden=128
)

optimizer = torch.optim.Adam(transformer.parameters(), 1e-3)

In [23]:
config = {
    "experiment_name": "Linear_Transformer_len_100_run_69",
    "dataset": "rosbank",
    "min_length": 20,
    "max_length": 100,
    "batch_size": 32,
    "type": "linear_transformer",
    "linear_transformer_params": {
        "feature_embeddings": {"small_group": (345, 64), "amount_rur_bin": (11, 64)}, 
        "linear_proj": 64,
        "n_head": 8, 
        "dim_feedforward": 128, 
        "dropout": 0.1, 
        "num_layers": 6, 
        "head_hidden": 128,
    },
    "lr": 1e-3,
    "n_epochs": 50,
    "warmup": 10,
    "device": "cuda",
    "save_path": "checkpoint"
}

In [24]:
def main(config):
    wandb.login()

    wandb.init(
        project="deep-learning-project",
        name=config["experiment_name"], 
        entity="aayugay99",
        config=config
    )

    if config["dataset"] == "rosbank":
        df = pd.read_csv('data/rosbank/train.csv')
        df['TRDATETIME'] = pd.to_datetime(df['TRDATETIME'], format='%d%b%y:%H:%M:%S')
        df = df.rename(columns={'cl_id':'client_id', 'MCC':'small_group', 'amount':'amount_rur'})
        
        mcc_to_id = {mcc: i+1 for i, mcc in enumerate(df['small_group'].unique())}

        df['amount_rur_bin'] = 1 + KBinsDiscretizer(10, encode='ordinal', subsample=None).fit_transform(df[['amount_rur']]).astype('int')
        df['small_group'] = df['small_group'].map(mcc_to_id)

    else:
        pass

    clients_train, clients_val_test = train_test_split(df["client_id"].unique(), test_size=0.2, random_state=42)
    clients_val, clients_test = train_test_split(clients_val_test, test_size=0.5, random_state=42)

    train_ds = TransactionDataset(
        df[lambda x: x["client_id"].isin(clients_train)], 
        id_col="client_id", 
        dt_col="TRDATETIME", 
        cat_cols=["small_group", "amount_rur_bin"],
        min_length=config["min_length"],
        max_length=config["max_length"],
        random_slice=True
    )

    val_ds = TransactionDataset(
        df[lambda x: x["client_id"].isin(clients_val)], 
        id_col="client_id", 
        dt_col="TRDATETIME", 
        cat_cols=["small_group", "amount_rur_bin"],
        min_length=config["min_length"],
        max_length=config["max_length"],
        random_slice=False
    )

    test_ds = TransactionDataset(
        df[lambda x: x["client_id"].isin(clients_test)], 
        id_col="client_id", 
        dt_col="TRDATETIME", 
        cat_cols=["small_group", "amount_rur_bin"],
        min_length=config["min_length"],
        max_length=config["max_length"],
        random_slice=False
    )

    train_loader = DataLoader(train_ds, batch_size=config["batch_size"], shuffle=True, collate_fn=transaction_collate_fn)
    val_loader = DataLoader(val_ds, batch_size=config["batch_size"], shuffle=False, collate_fn=transaction_collate_fn)
    test_loader = DataLoader(test_ds, batch_size=config["batch_size"], shuffle=False, collate_fn=transaction_collate_fn)

    transformer = LinearTransformerModel(**config["linear_transformer_params"])
    optimizer = torch.optim.Adam(transformer.parameters(), lr=config["lr"])

    train_model(
        transformer, 
        optimizer, 
        {"train": train_loader, "val": val_loader, "test": test_loader}, 
        n_epochs=config["n_epochs"],
        warmup=config["warmup"],
        device=config["device"],
        save_path=config["save_path"]
    )

    wandb.finish()

In [25]:
main(config)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: efadeev3 (aayugay99). Use `wandb login --relogin` to force relogin


628 sequences were filtered
77 sequences were filtered
70 sequences were filtered


...

  0%|          | 0/106 [00:00<?, ?it/s]

  0%|          | 0/106 [00:00<?, ?it/s]

  0%|          | 0/106 [00:00<?, ?it/s]

  0%|          | 0/106 [00:00<?, ?it/s]

Epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
Train loss,█▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train time,▇▄▁▁▁▂█▆▃▂▄▂▅▂▂▅▂▄▂▃▂▃▃▂▁▁▄▂▅▂▃▅▅▂▁▄▁▅▂▄
Val loss,█▅▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▂▁▁▂▂▂▂▂▂▂▂▂▂▂
Val time,▁▁▂▂▂▅▇▂▂▂█▃▂▂▁▂▅▁▂▅▂▇▂▂▂▂▂▂▂▂▆▂█▂▂▂▂▂▂▂
Epoch,50
Train loss,5.22903
Train time,4.94453
Val loss,5.35276
Val time,0.19716
